In [28]:
import glob
import re
import os
import pickle
import math
import numpy as np
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
# warnings.filterwarnings('default')

In [29]:
df = pd.read_csv("KaggleData/sp500_stocks.csv")
df.head()
companies = df["Symbol"].unique()
dates = df["Date"].unique()
rates = []
company_names = []
for c in companies:
    df_c = df.query("Symbol == @c")
    x = df_c["Adj Close"].values
    if np.isnan(x).any():
        print(f"Company {c} has no full length")
        continue
    company_names.append(c)
    dx = (x[1:] - x[:-1]) / x[:-1]
    rates.append(dx)
rates = np.array(rates)
print("Number of dates = ", len(dates))
print("Number of companies = ", len(company_names))

Company ABBV has no full length
Company ABNB has no full length
Company ALLE has no full length
Company AMCR has no full length
Company APTV has no full length
Company ANET has no full length
Company CZR has no full length
Company CARR has no full length
Company CTLT has no full length
Company CBOE has no full length
Company CDW has no full length
Company CHTR has no full length
Company CFG has no full length
Company CEG has no full length
Company CPAY has no full length
Company CTVA has no full length
Company DAY has no full length
Company FANG has no full length
Company DOW has no full length
Company ENPH has no full length
Company EPAM has no full length
Company ETSY has no full length
Company FTV has no full length
Company FOXA has no full length
Company FOX has no full length
Company GEHC has no full length
Company GEV has no full length
Company GNRC has no full length
Company GM has no full length
Company HCA has no full length
Company HPE has no full length
Company HLT has no fu

In [30]:
# binarize rates
ncompanies = rates.shape[0]
ndays = rates.shape[1]
average_rates = np.average(rates, axis=0)
binarized_rates = np.zeros((ndays, ncompanies), dtype="u1")
binarized_rates0 = np.zeros((ndays, ncompanies), dtype="u1")
for i in range(ndays):
    index = rates[:, i] > average_rates[i]
    binarized_rates[i, index] = 1
    index0 = rates[:, i] > 0
    binarized_rates0[i, index0] = 1  # corrected code!
# check diff binarized_rates and binarized_rates0
for i in range(ndays):
    ndiff = 0
    for j in range(ncompanies):
        if binarized_rates[i, j] != binarized_rates0[i, j]:
            ndiff += 1
    if ndiff > 0:
        print(f"Day {i} has {ndiff} different values")

Day 0 has 43 different values
Day 1 has 37 different values
Day 2 has 70 different values
Day 3 has 42 different values
Day 4 has 39 different values
Day 5 has 131 different values
Day 6 has 179 different values
Day 7 has 23 different values
Day 8 has 157 different values
Day 9 has 182 different values
Day 10 has 136 different values
Day 11 has 154 different values
Day 12 has 187 different values
Day 13 has 70 different values
Day 14 has 72 different values
Day 15 has 69 different values
Day 16 has 154 different values
Day 17 has 150 different values
Day 18 has 209 different values
Day 19 has 161 different values
Day 20 has 106 different values
Day 21 has 224 different values
Day 22 has 39 different values
Day 23 has 99 different values
Day 24 has 188 different values
Day 25 has 39 different values
Day 26 has 175 different values
Day 27 has 13 different values
Day 28 has 192 different values
Day 29 has 94 different values
Day 30 has 89 different values
Day 31 has 106 different values
D

In [31]:
# sample_data
[nsample, size] = binarized_rates.shape
sample_data = np.zeros((nsample * size * 2), dtype="u1")
for i in range(size):
    index = (
        np.arange(0, nsample * size * 2, size * 2, dtype=int)
        + i * 2
        + binarized_rates[:, i]
    )
    sample_data[index] = 1
sample_data = sample_data.reshape((nsample, size, 2))
# permute and divide into sample and test
nsample_h = nsample // 2
rng = np.random.default_rng(seed=20240419)
sample_data_perm = rng.permutation(sample_data, axis=0)
sample_data_h = sample_data_perm[:nsample_h, :, :]
test_data_h = sample_data_perm[nsample_h:, :, :]
print(nsample_h, nsample - nsample_h)
# save sample_data_h
data_name = "sp500h"
sample_name = "sample"
sample_data_h.tofile(f"{data_name}_{sample_name}.dat")
test_name = "test"
test_data_h.tofile(f"{data_name}_{test_name}.dat")
with open(f"{data_name}.des", "w") as f:
    f.write(f"size={size}\n")
    f.write(f"sample_name={sample_name}\n")
    f.write(f"test_name={test_name}\n")
    f.write("dtype=u1\n")

1794 1795


In [33]:
# sector_lists
df_company = pd.read_csv("KaggleData/sp500_companies.csv")
df_company.head()
sector_names = df_company["Sector"].unique()
sector_lists = dict()
i = 0
for s in sector_names:
    sector_lists[s] = i
    sector_lists[i] = s
    i += 1
with open("sp500_sector_lists.pkl", "wb") as f:
    pickle.dump(sector_lists, f)
# company_data
company_data = []
for c in company_names:
    df_c = df_company.query("Symbol == @c")
    if df_c.empty:
        print(f"Company {c} is not found in the company list")
        continue
    sector = df_c["Sector"].values[0]
    short_name = df_c["Shortname"].values[0]
    company_data.append([c, short_name, sector, sector_lists[sector]])
with open("sp500_company_data.pkl", "wb") as f:
    pickle.dump(company_data, f)